In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
df = pd.read_csv('Datasets\Microsoft_Azure_Predictive_Maintenance\PdM_telemetry.csv')
df.head()

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


In [3]:
failure = pd.read_csv('Datasets\Microsoft_Azure_Predictive_Maintenance\PdM_failures.csv')
failure

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4
...,...,...,...
756,2015-11-29 06:00:00,99,comp3
757,2015-12-14 06:00:00,99,comp4
758,2015-02-12 06:00:00,100,comp1
759,2015-09-10 06:00:00,100,comp1


In [4]:
ID = failure['machineID'].unique()
count = {}
for i in ID:
    count[i] = failure[failure['machineID']==i].shape[0]

In [5]:
df1 = df[df['machineID']==1]
df1.head()

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


In [6]:
failure[failure['machineID']==1].head()

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4


In [7]:
d = df[:99]
m = d[:100]
m.shape

(99, 6)

In [8]:
data = []
label = []

In [9]:
df[df['machineID']==ID[0]]

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511
...,...,...,...,...,...,...
8756,2016-01-01 02:00:00,1,153.724780,450.254337,85.197443,43.698137
8757,2016-01-01 03:00:00,1,172.418099,375.701907,95.568265,42.111661
8758,2016-01-01 04:00:00,1,174.588145,460.505353,95.483913,33.293572
8759,2016-01-01 05:00:00,1,184.224228,438.823315,102.694358,46.609515


In [24]:
for i in ID:
    data = []
    label = []
    machine = df[df['machineID']==i]
    failure_machine = failure[failure['machineID']==i]
    machine.drop(columns=['machineID'], inplace=True)
    for timestamp in failure_machine['datetime']:
        filtered_time = machine[machine['datetime']<=timestamp]
        machine.drop(machine[machine['datetime']<=timestamp].index, inplace=True)
        filtered_time.drop(columns=['datetime'], inplace=True)
        filtered = filtered_time[-100:]
        if filtered.shape[0] == 100:
            np_array = filtered.to_numpy()
            tensor = torch.tensor(np_array, dtype=torch.float).unsqueeze(0)
            data.append(tensor)
            label.append(torch.tensor([1]))
        filtered_time.drop(filtered.index)

        filtered = filtered_time[-100:]
        # for limiting the number of non failing componnents
        j = 0
        while filtered.shape[0] == 100 and j < 10:
            np_array = filtered.to_numpy()
            tensor = torch.tensor(np_array, dtype=torch.float).unsqueeze(0)
            data.append(tensor)
            label.append(torch.tensor([0]))
            filtered_time.drop(filtered.index)
            filtered = filtered_time[-100:]
            j += 1

    data_tensor = torch.cat(data, dim=0)
    label_tensor = torch.cat(label, dim=0)
    torch.save(data_tensor,'data_{}.pt'.format(i))
    torch.save(label_tensor, 'label_{}'.format(i))


C:\Users\Asus\AppData\Local\Temp\ipykernel_3388\998107595.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  machine.drop(columns=['machineID'], inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_3388\998107595.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  machine.drop(machine[machine['datetime']<=timestamp].index, inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_3388\998107595.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-